## Imports

In [23]:
from bs4 import BeautifulSoup
import pandas as pd

## Scrape Data

In [24]:
page = '/Users/afarence/Documents/coding_projects/trd-news-interactive-maps/NYCskyScrapers/List_ Skyscrapers Under Construction or Planned in New York – The Tower Info.html'

In [25]:
with open(page, "r") as file:
    html = file.read()

soup = BeautifulSoup(html, "html.parser")

In [26]:
soup


<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8" content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://thetowerinfo.com/xmlrpc.php" rel="pingback"/>
<title>List: Skyscrapers Under Construction or Planned in New York – The Tower Info</title>
<link href="//secure.gravatar.com" rel="dns-prefetch">
<link href="//cdn.jsdelivr.net" rel="dns-prefetch">
<link href="//s.w.org" rel="dns-prefetch"/>
<link href="https://thetowerinfo.com/feed/" rel="alternate" title="The Tower Info » Feed" type="application/rss+xml"/>
<link href="https://thetowerinfo.com/comments/feed/" rel="alternate" title="The Tower Info » Comments Feed" type="application/rss+xml"/>
<link href="https://thetowerinfo.com/nyc-skyscrapers-under-construction-proposed/feed/" rel="alternate" title="The Tower Info » List: Skyscrapers Under Construction or Plann